In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Link notebook with google drive and access data from your personal Gdrive
from google.colab import drive
#Mount Google Drive
drive.mount('/content/gdrive')

#Set the data path for dataset and model location (ex: model_loc = "/content/gdrive/My Drive/Dataset/")
dataset_dir = "/content/gdrive/My Drive/Bio Modelling/data/"
model_loc = "/content/gdrive/My Drive/Bio Modelling/data/"

print(os.listdir(dataset_dir))

Mounted at /content/gdrive
['BRCA_label.csv', 'BRCA_miRNA.csv', 'BRCA_DNA_Methylation.csv', 'BRCA_mRNA.csv', 'BRCA_Copy Nunber Variation.csv']


In [3]:
data = pd.read_csv(dataset_dir+'BRCA_miRNA.csv',index_col=0)
data

,TCGA.3C.AAAU.01,TCGA.3C.AALI.01,TCGA.3C.AALJ.01,TCGA.3C.AALK.01,TCGA.5L.AAT0.01,TCGA.5T.A9QA.01,TCGA.A1.A0SB.01,TCGA.A1.A0SE.01,TCGA.A1.A0SF.01,TCGA.A1.A0SG.01,...,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01,TCGA.V7.A7HQ.01,TCGA.W8.A86G.01,TCGA.WT.AB41.01,TCGA.WT.AB44.01,TCGA.XX.A899.01,TCGA.XX.A89A.01,TCGA.Z7.A8R5.01,TCGA.Z7.A8R6.01
hsa.let.7a.1,0.068317,-0.301684,-0.150810,0.107831,0.395211,1.135212,1.813887,0.473642,-0.470612,0.791968,...,-3.038971,-0.474111,0.165775,0.706495,-0.086149,0.511958,1.225298,0.667662,-0.211878,0.474240
hsa.let.7a.2,0.068932,-0.318009,-0.122747,0.097594,0.412879,1.140183,1.790792,0.442224,-0.461217,0.841821,...,-3.037340,-0.477443,0.172506,0.706552,-0.074380,0.504240,1.219548,0.666012,-0.210525,0.488117
hsa.let.7a.3,0.073899,-0.301310,-0.126333,0.095545,0.418441,1.143605,1.822272,0.431312,-0.470734,0.805374,...,-3.017967,-0.474484,0.164726,0.727433,-0.102883,0.496009,1.210610,0.675998,-0.220135,0.481577
hsa.let.7b,0.524562,0.419859,-0.958939,0.615389,0.500594,1.159087,2.113821,0.812876,-0.129307,0.867411,...,-3.509653,-0.855498,-0.359797,0.997679,-0.342075,0.422382,0.050804,-0.102655,0.177655,-0.078426
hsa.let.7c,-1.656853,-0.715963,-0.971038,0.711952,0.426323,-1.058870,2.135373,1.031809,0.864211,0.071249,...,-2.454470,0.188937,-0.001077,1.302563,0.506728,1.028977,1.186182,1.534247,1.070328,-0.155655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hsa.mir.95,0.943346,-0.828843,-0.487207,-0.539524,-0.490655,1.716711,-0.311971,-0.207033,-0.411471,1.331634,...,-1.290823,-1.028967,1.079692,-0.906212,0.475039,-0.867968,-0.019299,-1.125255,-0.477013,0.266220
hsa.mir.96,1.552613,0.599794,1.261482,0.856983,1.188748,0.375870,0.297224,0.066368,0.268172,0.155349,...,1.077028,-0.538255,0.875546,0.880267,0.479055,0.988195,-1.079726,0.219581,0.480704,0.198107
hsa.mir.98,-0.143572,1.103662,1.940594,-0.831098,-0.885905,-0.922881,0.390134,-0.330246,-0.481707,0.490018,...,1.152087,-0.648284,0.562151,-0.826311,0.280432,0.034551,0.419871,0.277618,-0.187117,0.914841
hsa.mir.99a,-1.251868,-0.627587,-0.848552,0.936474,0.449284,-1.316179,2.106744,1.078635,1.255710,-0.062945,...,-2.406554,0.465270,0.281323,1.373466,0.740620,0.989033,1.595339,1.039647,1.008012,0.094512


In [4]:
label = pd.read_csv(dataset_dir+'BRCA_label.csv')
label

,Label
0,LumA
1,Her2
2,LumB
3,LumA
4,LumA
...,...
666,LumA
667,LumA
668,LumA
669,LumA


In [5]:
#replace label with numerical data
label['Label'].replace('Normal', 0 , inplace=True)
label['Label'].replace('LumA', 1 , inplace=True)
label['Label'].replace('LumB', 2 , inplace=True)
label['Label'].replace('Basal', 3 , inplace=True)
label['Label'].replace('Her2', 4 , inplace=True)

In [6]:
# Transpose data to match with label
data = data. T
data

,hsa.let.7a.1,hsa.let.7a.2,hsa.let.7a.3,hsa.let.7b,hsa.let.7c,hsa.let.7d,hsa.let.7e,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,...,hsa.mir.935,hsa.mir.937,hsa.mir.939,hsa.mir.940,hsa.mir.942,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b
TCGA.3C.AAAU.01,0.068317,0.068932,0.073899,0.524562,-1.656853,-0.038283,0.501125,-2.390084,-2.406331,-0.760042,...,1.795470,0.108421,0.179279,1.768897,0.226659,0.943346,1.552613,-0.143572,-1.251868,1.603022
TCGA.3C.AALI.01,-0.301684,-0.318009,-0.301310,0.419859,-0.715963,0.460975,-1.999304,-0.659788,-0.651943,-1.050266,...,-0.409131,0.569264,0.188875,-0.453994,0.804807,-0.828843,0.599794,1.103662,-0.627587,-1.506825
TCGA.3C.AALJ.01,-0.150810,-0.122747,-0.126333,-0.958939,-0.971038,0.866585,2.074809,1.080746,1.088164,0.523204,...,0.742000,1.952380,-1.226285,-0.281690,-0.314543,-0.487207,1.261482,1.940594,-0.848552,0.787948
TCGA.3C.AALK.01,0.107831,0.097594,0.095545,0.615389,0.711952,-0.454282,0.227441,-0.735552,-0.686623,-1.015715,...,-0.715237,0.119549,-0.340371,-0.586990,-0.364370,-0.539524,0.856983,-0.831098,0.936474,0.030739
TCGA.5L.AAT0.01,0.395211,0.412879,0.418441,0.500594,0.426323,-1.545556,-0.952282,-0.280966,-0.233746,-0.818574,...,-0.925821,-0.248950,-1.439120,-0.707823,-0.109361,-0.490655,1.188748,-0.885905,0.449284,-0.645458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.511958,0.504240,0.496009,0.422382,1.028977,-0.254668,0.528134,0.757211,0.741367,0.378499,...,-0.927603,-0.345687,0.556464,-0.716083,-1.510158,-0.867968,0.988195,0.034551,0.989033,-0.893453
TCGA.XX.A899.01,1.225298,1.219548,1.210610,0.050804,1.186182,-1.238797,0.879213,1.455206,1.460326,0.921697,...,0.232086,-0.066436,-0.521624,-1.400357,0.188031,-0.019299,-1.079726,0.419871,1.595339,-0.898431
TCGA.XX.A89A.01,0.667662,0.666012,0.675998,-0.102655,1.534247,0.305716,1.276369,1.253312,1.240593,0.556762,...,-0.638539,-0.896460,-0.418889,-0.438539,-1.424222,-1.125255,0.219581,0.277618,1.039647,-0.363449
TCGA.Z7.A8R5.01,-0.211878,-0.210525,-0.220135,0.177655,1.070328,-0.716526,-0.598473,-0.229774,-0.190906,-0.465549,...,-0.356020,-1.035048,0.003670,-0.570925,-0.979351,-0.477013,0.480704,-0.187117,1.008012,-0.572713


In [7]:
# Ensure the number of rows in data match with label
if data.shape[0] == label.shape[0]:
    # Add the new column to the transposed DataFrame
    data['Label'] = label['Label'].values
else:
    raise ValueError("The number of rows in df2 must match the number of columns in the transposed df1.")


In [8]:
data

,hsa.let.7a.1,hsa.let.7a.2,hsa.let.7a.3,hsa.let.7b,hsa.let.7c,hsa.let.7d,hsa.let.7e,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,...,hsa.mir.937,hsa.mir.939,hsa.mir.940,hsa.mir.942,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b,Label
TCGA.3C.AAAU.01,0.068317,0.068932,0.073899,0.524562,-1.656853,-0.038283,0.501125,-2.390084,-2.406331,-0.760042,...,0.108421,0.179279,1.768897,0.226659,0.943346,1.552613,-0.143572,-1.251868,1.603022,1
TCGA.3C.AALI.01,-0.301684,-0.318009,-0.301310,0.419859,-0.715963,0.460975,-1.999304,-0.659788,-0.651943,-1.050266,...,0.569264,0.188875,-0.453994,0.804807,-0.828843,0.599794,1.103662,-0.627587,-1.506825,4
TCGA.3C.AALJ.01,-0.150810,-0.122747,-0.126333,-0.958939,-0.971038,0.866585,2.074809,1.080746,1.088164,0.523204,...,1.952380,-1.226285,-0.281690,-0.314543,-0.487207,1.261482,1.940594,-0.848552,0.787948,2
TCGA.3C.AALK.01,0.107831,0.097594,0.095545,0.615389,0.711952,-0.454282,0.227441,-0.735552,-0.686623,-1.015715,...,0.119549,-0.340371,-0.586990,-0.364370,-0.539524,0.856983,-0.831098,0.936474,0.030739,1
TCGA.5L.AAT0.01,0.395211,0.412879,0.418441,0.500594,0.426323,-1.545556,-0.952282,-0.280966,-0.233746,-0.818574,...,-0.248950,-1.439120,-0.707823,-0.109361,-0.490655,1.188748,-0.885905,0.449284,-0.645458,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.511958,0.504240,0.496009,0.422382,1.028977,-0.254668,0.528134,0.757211,0.741367,0.378499,...,-0.345687,0.556464,-0.716083,-1.510158,-0.867968,0.988195,0.034551,0.989033,-0.893453,1
TCGA.XX.A899.01,1.225298,1.219548,1.210610,0.050804,1.186182,-1.238797,0.879213,1.455206,1.460326,0.921697,...,-0.066436,-0.521624,-1.400357,0.188031,-0.019299,-1.079726,0.419871,1.595339,-0.898431,1
TCGA.XX.A89A.01,0.667662,0.666012,0.675998,-0.102655,1.534247,0.305716,1.276369,1.253312,1.240593,0.556762,...,-0.896460,-0.418889,-0.438539,-1.424222,-1.125255,0.219581,0.277618,1.039647,-0.363449,1
TCGA.Z7.A8R5.01,-0.211878,-0.210525,-0.220135,0.177655,1.070328,-0.716526,-0.598473,-0.229774,-0.190906,-0.465549,...,-1.035048,0.003670,-0.570925,-0.979351,-0.477013,0.480704,-0.187117,1.008012,-0.572713,1


In [9]:
# print the total number of data with missing values across all columns
print(data.isna().sum().sum())

0


In [10]:
# print the total number of data with duplicated values across all columns
print(data.duplicated().sum())

0


In [11]:
print(data["Label"].value_counts()[0])
print(data["Label"].value_counts()[1])
print(data["Label"].value_counts()[2])
print(data["Label"].value_counts()[3])
print(data["Label"].value_counts()[4])

31
353
132
113
42


In [12]:
# Separate features and target
y = data['Label'].values
x = data.drop(['Label'],axis=1)

In [13]:
# Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
X_normalized = scaler.fit_transform(x)

# Apply the scaler to the DataFrame
x = pd.DataFrame(scaler.fit_transform(X_normalized), index = x.index, columns = x.columns)
x

,hsa.let.7a.1,hsa.let.7a.2,hsa.let.7a.3,hsa.let.7b,hsa.let.7c,hsa.let.7d,hsa.let.7e,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,...,hsa.mir.935,hsa.mir.937,hsa.mir.939,hsa.mir.940,hsa.mir.942,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b
TCGA.3C.AAAU.01,0.077127,0.074798,0.076423,0.285187,-0.530754,-0.014734,0.392527,-0.530835,-0.523447,-0.203173,...,0.205520,-0.387108,-0.236899,0.523473,-0.247222,0.175782,0.629658,0.128714,-0.284763,0.563647
TCGA.3C.AALI.01,-0.025803,-0.033109,-0.028039,0.251832,-0.232318,0.133965,-0.391993,-0.070587,-0.059396,-0.287028,...,-0.623367,-0.216862,-0.233333,-0.324011,-0.069232,-0.441474,0.322503,0.453843,-0.089802,-0.456717
TCGA.3C.AALJ.01,0.016169,0.021344,0.020677,-0.187414,-0.313224,0.254772,0.886277,0.392385,0.400877,0.167600,...,-0.190564,0.294092,-0.759165,-0.258319,-0.413837,-0.322482,0.535808,0.672014,-0.158809,0.296215
TCGA.3C.AALK.01,0.088120,0.082791,0.082449,0.314122,0.220596,-0.138636,0.306657,-0.090740,-0.068569,-0.277045,...,-0.738457,-0.382997,-0.429986,-0.374716,-0.429177,-0.340704,0.405412,-0.050510,0.398651,0.047769
TCGA.5L.AAT0.01,0.168065,0.170715,0.172346,0.277552,0.129999,-0.463662,-0.063486,0.030178,0.051221,-0.220085,...,-0.817632,-0.519129,-0.838248,-0.420784,-0.350669,-0.323682,0.512361,-0.064797,0.246503,-0.174096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44.01,0.200543,0.196193,0.193941,0.252636,0.321152,-0.079183,0.401001,0.306327,0.309146,0.125790,...,-0.818302,-0.554866,-0.096748,-0.423933,-0.781921,-0.455101,0.447710,0.175147,0.415065,-0.255465
TCGA.XX.A899.01,0.398986,0.395673,0.392892,0.134261,0.371015,-0.372297,0.511154,0.491990,0.499317,0.282738,...,-0.382282,-0.451704,-0.497334,-0.684815,-0.259114,-0.159509,-0.218913,0.275592,0.604414,-0.257099
TCGA.XX.A89A.01,0.243858,0.241307,0.244052,0.085374,0.481416,0.087723,0.635763,0.438287,0.441196,0.177296,...,-0.709620,-0.758334,-0.459161,-0.318118,-0.755465,-0.544714,0.199936,0.238510,0.430872,-0.081567
TCGA.Z7.A8R5.01,-0.000820,-0.003135,-0.005439,0.174672,0.334268,-0.216743,0.047523,0.043795,0.062552,-0.118084,...,-0.603398,-0.809531,-0.302150,-0.368591,-0.618506,-0.318931,0.284113,0.117363,0.420993,-0.150228


In [14]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [15]:
print(y_train.tolist().count(0))
print(y_train.tolist().count(1))
print(y_train.tolist().count(2))
print(y_train.tolist().count(3))
print(y_train.tolist().count(4))

20
247
98
78
26


In [ ]:
# Define the SVM model
model = SVC(kernel="linear")

# Define RFE with the SVM model, choosing a number of features to select
rfe = RFE(estimator=model, n_features_to_select=300)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Transform the training and test sets to include only the selected features
x_train_rfe = rfe.transform(X_train)
x_test_rfe = rfe.transform(X_test)

In [ ]:
# Convert the transformed arrays back to DataFrames
# Get the selected feature names
selected_features = X_train.columns[rfe.support_]

X_train_rfe = pd.DataFrame(x_train_rfe, columns=selected_features, index=X_train.index)
X_test_rfe = pd.DataFrame(x_test_rfe, columns=selected_features, index=X_test.index)

In [ ]:
# Define the architecture parameters
input_dim = X_train_rfe.shape[1]   # Number of features
hidden1_dim = 256  # Increased first hidden layer dimension
hidden2_dim = 128  # Increased second hidden layer dimension
encoding_dim = 100  # Size of the encoded representations

# Build the SDAE model
input_layer = Input(shape=(input_dim,))
hidden1 = Dense(hidden1_dim, activation='relu')(input_layer)
encoded1 = Dense(hidden2_dim, activation='relu')(hidden1)
encoded2 = Dense(encoding_dim, activation='relu')(encoded1)
decoded1 = Dense(hidden2_dim, activation='relu')(encoded2)
decoded2 = Dense(hidden1_dim, activation='relu')(decoded1)
output_layer = Dense(input_dim, activation='linear')(decoded2)

sdae = Model(inputs=input_layer, outputs=output_layer)

# Compile the SDAE model
sdae.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the SDAE
sdae.fit(X_train_rfe, X_train_rfe, epochs=250, batch_size=32, shuffle=True, validation_data=(X_test_rfe, X_test_rfe), callbacks=[early_stopping])

# Extract the encoder part for feature extraction
encoder = Model(inputs=input_layer, outputs=encoded2)

# Extract features using the encoder
X_train_encoded = encoder.predict(X_train_rfe)
X_test_encoded = encoder.predict(X_test_rfe)

Epoch 1/250
15/15 [==============================] - 5s 21ms/step - loss: 0.1266 - val_loss: 0.0981
Epoch 2/250
15/15 [==============================] - 0s 7ms/step - loss: 0.0918 - val_loss: 0.0853
Epoch 3/250
15/15 [==============================] - 0s 6ms/step - loss: 0.0806 - val_loss: 0.0773
Epoch 4/250
15/15 [==============================] - 0s 6ms/step - loss: 0.0738 - val_loss: 0.0727
Epoch 5/250
15/15 [==============================] - 0s 6ms/step - loss: 0.0691 - val_loss: 0.0701
Epoch 6/250
15/15 [==============================] - 0s 7ms/step - loss: 0.0654 - val_loss: 0.0673
Epoch 7/250
15/15 [==============================] - 0s 6ms/step - loss: 0.0619 - val_loss: 0.0656
Epoch 8/250
15/15 [==============================] - 0s 8ms/step - loss: 0.0589 - val_loss: 0.0639
Epoch 9/250
15/15 [==============================] - 0s 6ms/step - loss: 0.0562 - val_loss: 0.0623
Epoch 10/250
15/15 [==============================] - 0s 6ms/step - loss: 0.0539 - val_loss: 0.0612
Epoch 11

In [ ]:
# Define a classifier (example: simple MLP classifier)
num_classes = len(np.unique(y_train))  # Number of classes in the target

classifier = Sequential([
    Dense(64, activation='relu', input_dim=encoding_dim),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multiclass classification
])

# Compile and train the classifier
classifier.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with early stopping
classifier.fit(X_train_encoded, y_train, epochs=250, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/250
12/12 [==============================] - 5s 65ms/step - loss: 1.9045 - accuracy: 0.2613 - val_loss: 1.4113 - val_accuracy: 0.5319
Epoch 2/250
12/12 [==============================] - 0s 14ms/step - loss: 1.6417 - accuracy: 0.3627 - val_loss: 1.3588 - val_accuracy: 0.5213
Epoch 3/250
12/12 [==============================] - 0s 15ms/step - loss: 1.4883 - accuracy: 0.4107 - val_loss: 1.3162 - val_accuracy: 0.5213
Epoch 4/250
12/12 [==============================] - 0s 17ms/step - loss: 1.3785 - accuracy: 0.4587 - val_loss: 1.2900 - val_accuracy: 0.5213
Epoch 5/250
12/12 [==============================] - 0s 17ms/step - loss: 1.2924 - accuracy: 0.5253 - val_loss: 1.2597 - val_accuracy: 0.5426
Epoch 6/250
12/12 [==============================] - 0s 23ms/step - loss: 1.2485 - accuracy: 0.5227 - val_loss: 1.2030 - val_accuracy: 0.5638
Epoch 7/250
12/12 [==============================] - 0s 20ms/step - loss: 1.2268 - accuracy: 0.5520 - val_loss: 1.1610 - val_accuracy: 0.5957
Epoch 

In [ ]:
# Predict probabilities
y_pred_prob = classifier.predict(X_test_encoded)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

7/7 [==============================] - 0s 2ms/step
Accuracy: 0.7475247524752475
